In [1]:
import os
import json

import pandas as pd
from IPython.display import display, HTML

from lkae.retrieval.retrieve import get_retriever, retrieve_evidence
from lkae.utils.data_loading import pkl_dir, load_pkl, root_dir

import pyterrier as pt
import pyterrier.io as ptio
import pyterrier.pipelines as ptpipelines
from ir_measures import R, MAP    

if not pt.started():
    pt.init()

datasets = {}

# walk through the pkl directory and load all the datasets in one of its subdirectories
# load each dataset with its subdirectory name and filename as the key
# skip non-pkl files
for subdir in os.listdir(pkl_dir):
    if not os.path.isdir(os.path.join(pkl_dir, subdir)):
        continue            
    datasets[subdir] = {}
    for filename in os.listdir(os.path.join(pkl_dir, subdir)):
        if not filename.endswith('.pkl'):
            continue
        key = os.path.join(subdir, filename)
        datasets[subdir][filename.split('.')[0]] = load_pkl(os.path.join(pkl_dir, key))

dataset_name = f'English_{"dev"}'
qrel_filename = f'{"dev"}_qrels.txt'

ds = datasets[dataset_name]
print(ds.keys())

golden = ptio.read_qrels(os.path.join(root_dir, 'data', qrel_filename))

PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8



dict_keys(['nopre-nam-bio', 'nopre-nam-nobio', 'nopre-nonam-bio', 'nopre-nonam-nobio', 'pre-nam-bio', 'pre-nam-nobio', 'pre-nonam-bio', 'pre-nonam-nobio'])


In [2]:
# load each config and construct its retriever

retrievers = {}

with open('config.json', 'r') as file:
    configs = json.load(file)

    for config in configs['configs']:
        retriever_label = get_retriever(**config)
        retrievers[config['retriever_method']] = retriever_label

retrievers

{'tfidf': <lkae.retrieval.methods.tfidf.TFIDFRetriever at 0x2254c9730d0>,
 'bm25': <lkae.retrieval.methods.bm25.BM25Retriever at 0x2254c973040>,
 'ollama': <lkae.retrieval.methods.ollama_emb.OllamaEmbeddingRetriever at 0x2254c973d30>}

In [3]:
# then for every variation of the dataset in ds, run the experiment with each retriever and save the results
import time

out_dir = 'results'

data = []

for dataset_label in ds:
    for retriever_label in retrievers:
        start = time.time()

        retrieved_data = retrieve_evidence(ds[dataset_label][:], retrievers[retriever_label])

        pred = pd.DataFrame([[*d, retriever_label] for d in retrieved_data], columns=['qid', 'docno', 'rank', 'score', 'name']) 

        eval = ptpipelines.Evaluate(pred, golden, metrics = [R@5,MAP], perquery=False)
        r5, meanap = [v for v in eval.values()]

        score = r5

        wall_time = time.time() - start

        print(f'result for retrieval run - R@5: {r5:.4f} MAP: {meanap:.4f} with config\tretriever: {retriever_label};\tds: {dataset_label}, took {wall_time:.2f} seconds')
        
        data.append({
            'R5': r5,
            'MAP': meanap,
            'Retrieval_Method': retriever_label, 
            'DS_Settings': dataset_label,
            'Time (s)': wall_time,
        })

# Convert the list of dictionaries to a DataFrame
df_retrieval = pd.DataFrame(data)

df_retrieval.to_csv(f'{out_dir}/df_retrieval.csv')
print(f'saved df to {out_dir}/df_retrieval.csv')

# Display the DataFrame
display(df_retrieval.sort_values(by='R5', ascending=False))

result for retrieval run - R@5: 0.6796 MAP: 0.6139 with config	retriever: tfidf;	ds: nopre-nam-bio, took 0.28 seconds
result for retrieval run - R@5: 0.7674 MAP: 0.6303 with config	retriever: bm25;	ds: nopre-nam-bio, took 0.61 seconds
result for retrieval run - R@5: 0.4382 MAP: 0.3329 with config	retriever: ollama;	ds: nopre-nam-bio, took 458.83 seconds
result for retrieval run - R@5: 0.7498 MAP: 0.6401 with config	retriever: tfidf;	ds: nopre-nam-nobio, took 0.23 seconds
result for retrieval run - R@5: 0.7653 MAP: 0.6119 with config	retriever: bm25;	ds: nopre-nam-nobio, took 0.48 seconds
result for retrieval run - R@5: 0.3593 MAP: 0.2689 with config	retriever: ollama;	ds: nopre-nam-nobio, took 381.61 seconds
result for retrieval run - R@5: 0.7323 MAP: 0.6256 with config	retriever: tfidf;	ds: nopre-nonam-bio, took 0.29 seconds
result for retrieval run - R@5: 0.7653 MAP: 0.6225 with config	retriever: bm25;	ds: nopre-nonam-bio, took 0.64 seconds
result for retrieval run - R@5: 0.3747 MAP:

,R5,MAP,Retrieval_Method,DS_Settings,Time (s)
13,0.767368,0.669789,bm25,pre-nam-bio,0.599967
1,0.767368,0.630316,bm25,nopre-nam-bio,0.609895
4,0.765263,0.611930,bm25,nopre-nam-nobio,0.479733
7,0.765263,0.622456,bm25,nopre-nonam-bio,0.637696
19,0.765263,0.655965,bm25,pre-nonam-bio,0.645116
16,0.765263,0.642632,bm25,pre-nam-nobio,0.474036
15,0.751930,0.653807,tfidf,pre-nam-nobio,0.222736
3,0.749825,0.640053,tfidf,nopre-nam-nobio,0.227664
6,0.732281,0.625643,tfidf,nopre-nonam-bio,0.285602
18,0.732281,0.646485,tfidf,pre-nonam-bio,0.305325
